# Exercise 05 ANN for Classification - Assignment

<span style="color:red">**The submission deadline of this assignment is 09/04/2024 23:59. In addition to the executed Jupyter Notebook, please also submit the saved `.pth` file of the trained ANN on Learn.**</span>

## Pedagogy

This notebook contains both theoretical explanations and executable cells to execute your code.

When you see the <span style="color:red">**[TBC]**</span> (To Be Completed) sign, it means that you need to perform an action else besides executing the cells of code that already exist. These actions can be:
- Complete the code with proper comments
- Respond to a question
- Write an analysis
- etc.

## Requirement

In this assignment, you are required to build a multi-class classification model using ANN.

The problem to be solved is to recognize the hand-written digits:

![](https://jamesmccaffrey.files.wordpress.com/2020/10/uci_digits_0_to_9-1.jpg)

<span style="color:red">**[TBC]**</span> Please complete the following tasks:
- Build the data pipeline
- Create a multi-class classification ANN
- Train the ANN
- Save and load the trained model
- Make predictions and evaluation

### Import libraries

In [ ]:
# [TBC] complete your code here with proper comments


## Task 1. Build the data pipeline

The hand-written digits dataset contains images of hand-written digits: 10 classes where each class refers to a digit. Find more details in this [link](https://scikit-learn.org/stable/datasets/toy_dataset.html#digits-dataset).
- Each hand-written digit image contains $8\times8$ pixels, which has already been converted to an 1D array with 64 features, each feature represent the value of a pixel.
- If you want to display the image, you need to re-converted the 1D array back to the 2D array ($8\times8$) with `reshape()` and display with `plt.imshow()`.

<span style="color:red">**[TBC]**</span> You need to:
- Use `sklearn.datasets.load_digits()` to load the hand-written digits dataset
- Pre-process the dataset
    - Scale numerical features with [standard scaling](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
    - encode categorical label with [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
- Divide the dataset into the training, valiation, and test set
- Create `TensorDataset` instances to store the dataset
- Create `DataLoader` instance to wrap the dataset as iterable objects
    - Decide a proper batch size considering
        - The available memory of your computer
        - The desired number of batches in each epoch

In [ ]:
# [TBC] complete your code here with proper comments


## Task 2. Create a multi-class classification ANN

<span style="color:red">**[TBC]**</span> You need to:
- Create a custom class to define the structure of the network
    - Include batch normalization layers
    - Include dropout layers
- Create an instance of this class as the ANN

In [ ]:
# [TBC] complete your code here with proper comments


## Task 3. Train the ANN

<span style="color:red">**[TBC]**</span> You need to:
- Define a `train()` function
    - Implement L2 regularization
    - Implement learning rate decay
    - Implement early stopping
- Train the ANN
    - Adjust the training hyper-parameters if necessary to get a good result
- Plot the loss history of the training process
    - Batch & epoch loss on the train set
    - Batch & epoch loss on the validation set
    - Train & validation epoch loss

In [ ]:
# [TBC] complete your code here with proper comments


## Task 4. Save and load the trained model

<span style="color:red">**[TBC]**</span> You need to:
- Save the trained model
    - <span style="color:red">**Don't forget to include the saved file in your submission.**</span>
- Re-create the neural network and load the saved model

In [ ]:
# [TBC] complete your code here with proper comments


## Task 5. Make predictions and evaluation

<span style="color:red">**[TBC]**</span> You need to:
- Define the `test()` function
    - Output logits, probabilities and predictions
    - Output the loss value on the test set
- Make predictions on the test dataset
- Evaluate the performance
    - Use `sklearn.metrics.classification_report()`

In [ ]:
# [TBC] complete your code here with proper comments
